### Creating a test set from validation set ###

In [ ]:
import os

# --- config ---
VOC_PATH = "./data/VOCdevkit/VOC2012"
IMAGE_DIR = os.path.join(VOC_PATH, "JPEGImages")
MASK_DIR = os.path.join(VOC_PATH, "SegmentationClass")
SPLIT_DIR = os.path.join(VOC_PATH, "ImageSets", "Segmentation")

TRAIN_FILE = os.path.join(SPLIT_DIR, "train.txt")
VAL_FILE = os.path.join(SPLIT_DIR, "val.txt")
TEST_FILE = os.path.join(SPLIT_DIR, "test.txt")

# --- for checking ---
if not os.path.exists(VAL_FILE):
    raise FileNotFoundError("val.txt not found.")
if os.path.exists(TEST_FILE):
    raise FileExistsError("test.txt already exists.")

# --- Load id files ---
with open(TRAIN_FILE, "r") as f:
    train_ids = [line.strip() for line in f]

with open(VAL_FILE, "r") as f:
    val_ids = [line.strip() for line in f]

if len(val_ids) != 1449:
    raise ValueError(f"Expected 1449 val IDs, but found {len(val_ids)}")

# --- split val to val + test ---
test_ids = val_ids[-450:]
val_ids = val_ids[:-450]

# --- WRITE UPDATED FILES ---
with open(VAL_FILE, "w") as f:
    f.write("\n".join(val_ids) + "\n")
with open(TEST_FILE, "w") as f:
    f.write("\n".join(test_ids) + "\n")

# --- filter imgs nd masks to match train + val + test ---
final_ids = set(train_ids + val_ids + test_ids)
all_images = os.listdir(IMAGE_DIR)
all_masks = os.listdir(MASK_DIR)

deleted = 0
for img_file in all_images:
    img_id = os.path.splitext(img_file)[0]
    if img_id not in final_ids:
        os.remove(os.path.join(IMAGE_DIR, img_file))
        deleted += 1

for mask_file in all_masks:
    mask_id = os.path.splitext(mask_file)[0]
    if mask_id not in final_ids:
        os.remove(os.path.join(MASK_DIR, mask_file))
        deleted += 1

# --- FINAL CHECK ---
print(f"\ntest.txt created with {len(test_ids)} IDs.")
print(f"val.txt updated with {len(val_ids)} IDs.")
print(f"\n Final counts:")
print(f"Train IDs: {len(train_ids)}")
print(f"Val IDs:   {len(val_ids)}")
print(f"Test IDs:  {len(test_ids)}")
print(f"Total kept image-mask pairs: {len(final_ids)}")
print(f"Expected pairs to keep:      {len(train_ids) + len(val_ids) + len(test_ids)}")
print(f"Deleted {deleted} files (images + masks not in splits).")


### Creating Class Distribution Plot ###

In [ ]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from tqdm import tqdm
from dataset import PascalVOCDataset
import albumentations as A
from albumentations.pytorch import ToTensorV2

data_dir = "./data/VOCdevkit/VOC2012"
num_classes = 21
image_height, image_width = 256, 256

transform = A.Compose([
    A.Resize(height=image_height, width=image_width),
    ToTensorV2(),
], additional_targets={'mask': 'mask'})

image_dir = os.path.join(data_dir, "JPEGImages")
mask_dir = os.path.join(data_dir, "SegmentationClass")
train_split_file = os.path.join(data_dir, "ImageSets", "Segmentation", "train.txt")
test_split_file = os.path.join(data_dir, "ImageSets", "Segmentation", "val.txt")

train_dataset = PascalVOCDataset(image_dir, mask_dir, train_split_file, transform=transform)
test_dataset = PascalVOCDataset(image_dir, mask_dir, test_split_file, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

def count_pixels(loader):
    counts = torch.zeros(num_classes, dtype=torch.long)
    for _, mask in tqdm(loader):
        for c in range(num_classes):
            counts[c] += (mask == c).sum()
    return counts

train_counts = count_pixels(train_loader).numpy()
test_counts = count_pixels(test_loader).numpy()

index = np.arange(num_classes)
bar_width = 0.35

plt.figure(figsize=(12,6))
plt.bar(index - bar_width/2, train_counts, bar_width, label='Train')
plt.bar(index + bar_width/2, test_counts, bar_width, label='Test')
plt.yscale('log')
plt.xlabel('Class Index')
plt.ylabel('Pixel Count (log scale)')
plt.title('Class Pixel Distribution (Train vs Test)')
plt.xticks(index)
plt.legend()
plt.tight_layout()
plt.savefig('class_distribution.png')


### Unique labels in train set ###

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm import tqdm

# Path to your train.txt split file
split_file = "/content/data/VOCdevkit/VOC2012/ImageSets/Segmentation/train.txt"  # update this
mask_dir = "/content/data/VOCdevkit/VOC2012/SegmentationClass"  # update this

# Read all image ids
with open(split_file, 'r') as f:
    image_ids = [line.strip() for line in f if line.strip()]

unique_labels = set()

for img_id in tqdm(image_ids):
    mask_path = os.path.join(mask_dir, f"{img_id}.png")
    mask = np.array(Image.open(mask_path))
    unique_labels.update(np.unique(mask).tolist())

print("Unique labels found in train masks:", sorted(unique_labels))


### Checking 1 image class no. ###

In [ ]:
import os
import numpy as np
from PIL import Image 


data_dir = "/content/data/VOCdevkit/VOC2012" 

mask_dir = os.path.join(data_dir, "SegmentationClass")
train_split_file = os.path.join(data_dir, "ImageSets", "Segmentation", "train.txt")

# Read a list of mask filenames from the train split file
mask_filenames = []
with open(train_split_file, 'r') as f:
    for line in f:
        
        image_id = line.strip()
        mask_filenames.append(f"{image_id}.png") # Masks are typically .png

if not mask_filenames:
    print(f"No mask filenames found in {train_split_file}. ")
else:
    # Load the first mask found to inspect its class numbers
    sample_mask_filename = mask_filenames[0]
    sample_mask_path = os.path.join(mask_dir, sample_mask_filename)

    if os.path.exists(sample_mask_path):
        print(f"Loading sample mask: {sample_mask_path}")
        mask_image = Image.open(sample_mask_path).convert('L') # Convert to grayscale 
        mask_array = np.array(mask_image)

        unique_classes = np.unique(mask_array)
        print(f"Unique class numbers in sample mask '{sample_mask_filename}': {unique_classes}")
        print(f"Minimum class number: {np.min(unique_classes)}")
        print(f"Maximum class number: {np.max(unique_classes)}")

        # Check for 255 specifically
        if 255 in unique_classes:
            print("Class 255 (ignore index) is present in this sample mask.")
    else:
        print(f"Sample mask not found at: {sample_mask_path}.")

In [ ]:
import os
import numpy as np
from PIL import Image # Pillow for loading images
from collections import Counter
from tqdm import tqdm # For progress bar

# Assume your data_dir is the same as in train.py
data_dir = "/content/data/VOCdevkit/VOC2012" # Replace if different

mask_dir = os.path.join(data_dir, "SegmentationClass")
train_split_file = os.path.join(data_dir, "ImageSets", "Segmentation", "train.txt")

print(f"Checking class imbalance for masks in: {mask_dir}")
print(f"Using mask list from: {train_split_file}")

all_pixel_counts = Counter()

# Read a list of mask filenames from the train split file
mask_filenames = []
try:
    with open(train_split_file, 'r') as f:
        for line in f:
            image_id = line.strip()
            mask_filenames.append(f"{image_id}.png")
except FileNotFoundError:
    print(f"Error: Train split file not found at {train_split_file}")
    exit()

if not mask_filenames:
    print("No mask filenames found to process. Exiting.")
    exit()

# Iterate through all masks with a progress bar
for filename in tqdm(mask_filenames, desc="Counting pixels per class"):
    mask_path = os.path.join(mask_dir, filename)
    if not os.path.exists(mask_path):
        # print(f"Warning: Mask not found for {filename}. Skipping.")
        continue # Skip if file doesn't exist

    try:
        mask_image = Image.open(mask_path).convert('L') # Convert to grayscale
        mask_array = np.array(mask_image)

        # Count pixel occurrences in this mask
        # np.bincount is efficient for integer counts
        # Ensure max value for bincount is large enough (e.g., 255)
        counts = np.bincount(mask_array.flatten(), minlength=256) # Max possible class ID is 255

        # Add to overall counts
        for class_id, count in enumerate(counts):
            if count > 0:
                all_pixel_counts[class_id] += count

    except Exception as e:
        print(f"Error processing mask {filename}: {e}")
        continue

print("\n--- Class Pixel Distribution ---")
total_pixels = sum(all_pixel_counts.values())

if total_pixels == 0:
    print("No pixels counted. Make sure masks are loaded correctly.")
else:
    # Sort by class ID for cleaner output
    sorted_class_ids = sorted(all_pixel_counts.keys())

    for class_id in sorted_class_ids:
        count = all_pixel_counts[class_id]
        percentage = (count / total_pixels) * 100
        label_name = f"Class {class_id}"
        if class_id == 0:
            label_name = "Class 0 (Background)"
        elif class_id == 255:
            label_name = "Class 255 (Ignore/Void)"
        print(f"{label_name: <25}: {count: <10} pixels ({percentage:.4f}%)")

    # Optionally, calculate percentage for actual training classes (excluding 0 and 255)
    training_pixels_sum = 0
    for class_id in sorted_class_ids:
        if class_id != 0 and class_id != 255:
            training_pixels_sum += all_pixel_counts[class_id]

    if training_pixels_sum > 0:
        print("\n--- Distribution of Actual Foreground Classes (Excluding Background and Ignore) ---")
        for class_id in sorted_class_ids:
            if class_id != 0 and class_id != 255:
                count = all_pixel_counts[class_id]
                percentage = (count / training_pixels_sum) * 100
                print(f"Class {class_id: <15}: {count: <10} pixels ({percentage:.4f}%)")
    else:
        print("\nNo foreground classes (excluding 0 and 255) found or counted.")